In [1]:
%matplotlib inline
from PIL import Image
import numpy
import os
import re
import random
from resizeimage import resizeimage
import matplotlib.pyplot as plt
from scipy import sparse
from sklearn.decomposition import PCA
import copy
import matplotlib.image as mpimg
from sklearn.decomposition import RandomizedPCA

In [2]:
# resize images
# resizeImages('../data/yalefaces/', 'subject01')
def resizeImagesToDest(source, dest, prefix, height, noiseAlgo=None):    
    for root, dirs, files in os.walk(source):
        for name in files:
            if re.match(r'^'+prefix+'.*gif$', name, re.I):
                img = Image.open(os.path.join(source, name))
                (origWidth, origHeight) = img.size
                if (height == origHeight):
                    resizedImg = img
                else:
                    resizedImg = resizeimage.resize_height(img, height)
                    
                (width, height) = resizedImg.size
                if noiseAlgo:
                    noisyImg = noiseAlgo(numpy.asarray(resizedImg))
                    Image.fromarray(numpy.uint8(noisyImg.reshape(height,width))).save(dest+name)
                else:
                    resizedImg.save(dest+name)

def resizeImages(source, prefix):
    resizeImagesToDest(source, 'yalefaces_resized/', prefix, 50)
                
# images into array: http://stackoverflow.com/questions/13550376/pil-image-to-array-numpy-array-to-array-python
def readImagesIntoMatrix(path, prefix):
    images = None
    for root, dirs, files in os.walk(path):
        for name in files:
            if re.match(r'^'+prefix+'.*gif$', name, re.I):
                img = Image.open(path + name)                
                (width, height) = img.size
                imgArr = numpy.asarray(img).reshape(height * width)
                if images is not None:
                    images = numpy.vstack([images, imgArr])
                else:
                    images = imgArr
    return images

def saveImages(path, images, width, height):
    for i in range(0, len(images)):
        Image.fromarray(numpy.uint8(images[i]).reshape(height, width)).save(path+str(i)+'.gif')
        
def addGaussianNoise(image):
    noisyImage = copy.deepcopy(image)
    mu = noisyImage.mean()
    std = noisyImage.std()
    noise = 0.2 * numpy.random.normal(mu, std, noisyImage.shape)
    for i in range(len(image)):
        for j in range(len(image[0])):
            val = image[i][j]
            if val + noise[i][j] >= 255:
                noisyImage[i][j] = 255
            elif val + noise[i][j] <= 0:
                noisyImage[i][j] = 0
            else:
                noisyImage[i][j] = val + noise[i][j] 
    
    return noisyImage  
    
def addSparseNoise(image):
    noisyImage = copy.deepcopy(image)
    for i in range(len(image)):
        for j in range(len(image[0])):
            rand = random.random()
            if (rand <= 0.01):
                noisyImage[i][j] = 0
    
    return noisyImage

In [3]:
resizeImages('yalefaces/', 'yale.*')
imageList = readImagesIntoMatrix('yalefaces_resized/', '.*').T

print("Images %s" % imageList)

Image.fromarray(numpy.uint8(imageList.T[0].reshape(50,44))).save("secondyaletest/testsaveimage1.gif")

Images [[155   4 135 ..., 161  84   5]
 [152   4 135 ..., 162  84   5]
 [136   4 120 ..., 166  88   5]
 ..., 
 [ 11 145   6 ...,   6   4 163]
 [ 11 141   6 ...,   6   6 178]
 [ 12 136   7 ...,  13   8 187]]


In [4]:
(width, height) = 44, 50
# average Image
#print('Num images %s' % (imageList.shape,))
avgImg = imageList.mean(axis=1)
print('Average image %s' % (avgImg))
normalizedImageList = imageList - avgImg.reshape(width*height,1)*numpy.ones((1,2414))
Image.fromarray(numpy.uint8(avgImg.reshape(height,width))).save('secondyaletest/avgImg.gif')

Average image [ 59.02526926  62.63835957  64.9908865  ...,  49.28583264  46.52734051
  40.11267606]


In [5]:
def normalize_face(vector):
    minValue = numpy.amin(vector)
    maxValue = numpy.amax(vector)
    scale = (255./(maxValue-minValue))
    return (vector - minValue)*scale

# Fit with PCA for 90% of variance
pca = PCA(0.9).fit_transform(normalizedImageList)
print('With PCA there were %s eigenfaces that captures 90pct of the variance' % len(pca.T))

for i in range(len(pca.T)):
    eigv = pca[:,i].reshape(1,width * height).T
    eigenface = normalize_face(eigv)
    Image.fromarray(numpy.uint8(eigenface.reshape(height,width))).save('secondyaletest/eigenface%s.gif' % i)

With PCA there were 33 eigenfaces that captures 90pct of the variance


In [25]:
from numpy.linalg import norm, svd
from scipy.io import loadmat, savemat

def inexact_augmented_lagrange_multiplier(X, lmbda=.01, tol=1e-3,
                                          maxiter=100, verbose=True):
    """
    Inexact Augmented Lagrange Multiplier
    """
    Y = X
    norm_two = norm(Y.ravel(), 2)
    norm_inf = norm(Y.ravel(), numpy.inf) / lmbda
    dual_norm = numpy.max([norm_two, norm_inf])
    Y = Y / dual_norm
    A = numpy.zeros(Y.shape)
    E = numpy.zeros(Y.shape)
    dnorm = norm(X, 'fro')
    mu = 1.25 / norm_two
    rho = 1.5
    sv = 10.
    n = Y.shape[0]
    itr = 0
    while True:
        if verbose:
            print('Iterations %s' % itr)
        Eraw = X - A + (1 / mu) * Y
        Eupdate = numpy.maximum(Eraw - lmbda / mu, 0) + numpy.minimum(Eraw + lmbda / mu, 0)
        U, S, V = svd(X - Eupdate + (1 / mu) * Y, full_matrices=False)
        svp = (S > 1 / mu).shape[0]
        if svp < sv:
            sv = numpy.min([svp + 1, n])
        else:
            sv = numpy.min([svp + round(.05 * n), n])
        Aupdate = numpy.dot(numpy.dot(U[:, :svp], numpy.diag(S[:svp] - 1 / mu)), V[:svp, :])
        A = Aupdate
        E = Eupdate
        Z = X - A - E
        Y = Y + mu * Z
        mu = numpy.min([mu * rho, mu * 1e7])
        itr += 1
        if ((norm(Z, 'fro') / dnorm) < tol) or (itr >= maxiter):
            break
    if verbose:
        print("Finished at iteration %d" % (itr))  
    return A,E

In [26]:
A, E = inexact_augmented_lagrange_multiplier(normalizedImageList)

Iterations 0
Iterations 1
Iterations 2
Iterations 3
Iterations 4
Iterations 5
Iterations 6
Iterations 7
Iterations 8
Iterations 9
Iterations 10
Iterations 11
Iterations 12
Iterations 13
Iterations 14
Iterations 15
Iterations 16
Iterations 17
Iterations 18
Iterations 19
Iterations 20
Iterations 21
Iterations 22
Iterations 23
Iterations 24
Iterations 25
Iterations 26
Iterations 27
Iterations 28
Finished at iteration 29


In [34]:
Image.fromarray(numpy.uint8((A.T[0] + avgImg).reshape(50,44))).save("secondyaletest/testsaveimage1afterrpca.gif")

print('Error %s' % E)

Error [[ 29.47154864   1.38587638  52.26933902 ...,  27.94830662  -7.23200304
   -1.71289954]
 [ 26.59521867   1.68307986  48.22074076 ...,  25.5982466   -7.22357765
   -2.48747389]
 [ 12.79967237   0.39494452  31.90784184 ...,  30.23312859  -1.76243837
   -3.45600502]
 ..., 
 [  0.57668763  21.04121044  -4.57883251 ...,  -3.64121018  -3.39215507
   53.56857782]
 [ -0.34770782  23.64493797  -5.53346484 ...,  -2.79193018  -2.10040627
   71.45596794]
 [  0.81938153  37.3198014   -4.88200021 ...,   3.73356815  -0.67882753
   96.40086592]]


In [13]:

# Fit with PCA for 90% of variance
rpca = PCA(0.9).fit_transform(A)

print('After RPCA there were %s eigenfaces that captures 90pct of the variance' % len(rpca.T))

#Let's make images for the same number of eigenfaces we used originally
rpca = PCA(len(pca.T)).fit_transform(A)

for i in range(len(rpca.T)):
    eigv = rpca[:,i].reshape(1,width * height).T
    eigenface = normalize_face(eigv)
    Image.fromarray(numpy.uint8(eigenface.reshape(height,width))).save('secondyaletest/rpca-eigenface%s.gif' % i)

After RPCA there were 3 eigenfaces that captures 90pct of the variance


In [35]:
import matplotlib.pyplot as plt
import warnings

def makeScatterPlot(xdata, ydata, title, xlabel, ylabel, outputFileName, marker):
  xdata = numpy.array(xdata)
  ydata = numpy.array(ydata)
  # to catch "falling back to Agg" warning
  with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    fig = plt.figure()
    fig.suptitle(title)
    plt.semilogy(xdata, ydata, marker)
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)
    plt.savefig(outputFileName, format = 'png')
    plt.close()

In [56]:
xdata = [0.0]*len(pca.T)
ydata = [0.0]*len(pca.T)

from sklearn.preprocessing import normalize

pca_normalized = normalize(pca, axis=0, norm='l2')
rpca_normalized = normalize(rpca, axis=0, norm='l2')

for i in range(len(pca.T)):
  xdata[i] = i
  dotting_together = pca_normalized[:,:i+1].T.dot(rpca_normalized[:,:i+1])
  # Grr figuring out a norm here is really difficult.
  #ydata[i] = numpy.linalg.norm(dotting_together, 'fro')

print('The cosine similarity between eigenvectors %s' % ydata)

makeScatterPlot(xdata, ydata, "Cosine similarity between eigenvectors found in PCA and RPCA", "Eigenvector", "Cosine Similarity", "attempt.png", "o")


The cosine similarity between eigenvectors [0.99974848648990577, 1.4090758613596432, 1.7249891586550608, 1.7667496307611807, 1.9962224496813032, 2.3923968583243593, 2.5873310911137666, 2.7830173291257023, 2.9351325247263222, 3.0492507294843443, 3.2633042867571764, 3.3338095965466779, 3.51493257140746, 3.5908991409229207, 3.7149634562028866, 3.8845933849978738, 3.9609731934126877, 4.0503913700186249, 4.1263143205246964, 4.2783818439245724, 4.3592841390910975, 4.4073140696910995, 4.5729117622221347, 4.6297767943347754, 4.7149564756365629, 4.7635082179352501, 4.8670029905215779, 4.9842396693937063, 5.0579712317591934, 5.1293946225119145, 5.197892582150673, 5.2689198340706929, 5.3630979527487348]
